# Fine-tune Gemma-2-2b for Code Audit (Kaggle T4)

**Stable Production Edition (2026)**

| | |
|---|---|
| 🤖 Model | `unsloth/gemma-2-2b-it-bnb-4bit` |
| ⚡ Framework | Unsloth (Stable GitHub Version) |
| ⏱️ Runtime | ~30 min |


## 1. Install Dependencies (Nuclear Clean)

In [ ]:
# KAGGLE UNSLOTH STABLE INSTALL (2026)
import subprocess

def run_pip(cmd, msg):
    print(msg)
    subprocess.run(f"pip install -q --no-cache-dir {cmd}", shell=True, capture_output=True)

print("="*60)
print("🚀 CLEANING & INSTALLING STABLE STACK")
print("="*60)

subprocess.run("pip uninstall -y torchao unsloth unsloth_zoo transformers -q", shell=True)

run_pip("fsspec==2024.9.0 datasets==4.2.0 huggingface_hub>=0.23.0", "[1/4] Installing base deps...")
run_pip("peft accelerate bitsandbytes trl transformers>=4.45.0", "[2/4] Installing transformers 4.45+...")
run_pip('"unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"', "[3/4] Installing Unsloth...")
run_pip("xformers==0.0.28.post3", "[4/4] Installing xformers...")

print("\n✅ DONE! RESTART KERNEL NOW! Then run Cell 2.")


## 2. Load Model

**Crucial:** Import `unsloth` before anything else.

In [ ]:
import unsloth  # Must be first!
import torch
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-2b-it-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)
print("✅ Model loaded!")


## 3. Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
model.print_trainable_parameters()


## 4. Load Dataset

In [ ]:
from datasets import load_dataset
import glob

for p in ["/kaggle/input/audit-dataset/audit_dataset.jsonl",
          "/kaggle/input/*/audit_dataset.jsonl",
          "/kaggle/input/*/*.jsonl"]:
    m = glob.glob(p)
    if m: dataset_path = m[0]; break
else: raise FileNotFoundError("Upload audit_dataset.jsonl!")

dataset = load_dataset("json", data_files=dataset_path, split="train")
ALPACA = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
\n### Instruction:\n{}\n\n### Response:\n{}"""
EOS = tokenizer.eos_token
dataset = dataset.map(lambda x: {"text": [ALPACA.format(i, o) + EOS for i, o in zip(x["instruction"], x["output"])]}, batched=True)
print(f"✅ Loaded {len(dataset)} examples")


## 5. Fine-tune (~30 min)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    args=TrainingArguments(
        output_dir="./outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        seed=3407,
        report_to="none",
    ),
)
trainer.train()


## 6. Save Model

In [ ]:
model.save_pretrained("audit-gemma-v1")
tokenizer.save_pretrained("audit-gemma-v1")
print("✅ Saved to audit-gemma-v1/")
